In [35]:
!pip install pytorch-lightning

# Basic CNN

In [133]:
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD

def acc(output, target, topk=(1,)):
    """generating the specified value of k """
    max_a = max(topk)
    size_batch = target.size(0)

    _, pred_a = output.topk(max_a, 1, True, True)
    pred_a = pred_a.t()
    just = pred_a.eq(target.view(1, -1).expand_as(pred_a))
    res_a = []

    for k in topk:
        just_k = just[:k].view(-1).float().sum(0)
        res_a.append(just_k.mul_(100.0 / size_batch))
    return res_a

class basic_cnn_(pl.LightningModule):
    def __init__(self):
          # Defining the model
        super(basic_cnn_,self).__init__()
          # 1st  2D-convolutional-layer has input  32 is output of feature size and kernel size being 3
        
        self.con_1 = nn.Conv2d(3, 32, 3)
          # 2nd  2D-convolutional-layer has input 32 from the above layer
        self.con_2 = nn.Conv2d(32, 64, 3)
        
         # 1st layer connects
        self.fc_1 = nn.Linear(57600,256)

         # 2nd layer connects
        self.fc_2 = nn.Linear(256,128)

         # 3rd layer connects gives 101 output 
        self.out = nn.Linear(128,101)
        self.lr = 0.01
         # applying cross entropy
        self.loss = nn.CrossEntropyLoss()
        
        
           
    def forward(self, x):
        # defining the forward pass
        size_batch, _, _, _ = x.size()
        x = F.relu(self.con_1(x))
        x = F.relu(self.con_2(x))
        x = F.max_pool2d(x, 2) 
        x = x.view(size_batch,-1)
        x = F.relu(self.fc_1(x))
        x = F.relu(self.fc_2(x))
        return self.out(x)
    
    def configure_optimizers(self):
         #  Optimizer being defined
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # training loop defined
        a, b = train_batch
        logits = self.forward(a)
        train_loss = self.loss(logits,b)
        print("Training loss = ", train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # validation loops defined
        a, b = valid_batch
        logits = self.forward(a)
        valid_loss = self.loss(logits,b)
        print("Validation loss = ", valid_loss)
        valid_pred = acc(logits, b)
        self.log("val_accuracy", valid_pred[0])
        self.log("val_loss", valid_loss)
        return valid_loss
    
    def test_step(self, test_batch, batch_idx):
         # testing steps being defined
        l_test=[]
        a, b = test_batch
        logits = self.forward(a)
        test_loss = self.loss(logits,b)
        l_test.append(test_loss)
        test_pred = acc(logits, b)
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [37]:
!pip install torchvision

In [38]:
pip install lightning-bolts


Note: you may need to restart the kernel to use updated packages.


In [134]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(64),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.229, 0.224, 0.225])])
        
        # Preparing data
        print("[-INFORMATION-] load - FOOD 101 dataset...")

      # spliting the train_data, val_data, test_data
        self.train_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="test", download=True, transform=transform)
        print('[-INFORMATION-] data loaded successful')
     #defining train data loader
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)
     #defining value data loader
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
     #defining test data loader
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)

In [135]:
 # model object being created
clfn_basic_cnn = basic_cnn_()
 #data module object created
Food101 = Food101()
 # trainer object being created
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [136]:
 # fit method for trainer
trainer.fit(clfn_basic_cnn, Food101)

[-INFORMATION-] load - FOOD 101 dataset...



  | Name  | Type             | Params
-------------------------------------------
0 | con_1 | Conv2d           | 896   
1 | con_2 | Conv2d           | 18.5 K
2 | fc_1  | Linear           | 14.7 M
3 | fc_2  | Linear           | 32.9 K
4 | out   | Linear           | 13.0 K
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
14.8 M    Trainable params
0         Non-trainable params
14.8 M    Total params
59.245    Total estimated model params size (MB)


[-INFORMATION-] data loaded successful


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\syeda\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


...

Training: 0it [00:00, ?it/s]

Training loss =  tensor(4.6190, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6412, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6070, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6115, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6214, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6167, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5956, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5848, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6057, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6010, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6240, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5993, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6149, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5984, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6088, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6254, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6282, grad_fn=

Validation: 0it [00:00, ?it/s]

Validation loss =  tensor(4.5309)
Validation loss =  tensor(4.4306)
Validation loss =  tensor(4.4113)
Validation loss =  tensor(4.5198)
Validation loss =  tensor(4.3371)
Validation loss =  tensor(4.3287)
Validation loss =  tensor(4.4796)
Validation loss =  tensor(4.3966)
Validation loss =  tensor(4.5522)
Validation loss =  tensor(4.4626)
Validation loss =  tensor(4.4217)
Validation loss =  tensor(4.4398)
Validation loss =  tensor(4.3845)
Validation loss =  tensor(4.4598)
Validation loss =  tensor(4.3663)
Validation loss =  tensor(4.4197)
Validation loss =  tensor(4.4661)
Validation loss =  tensor(4.5940)
Validation loss =  tensor(4.4105)
Validation loss =  tensor(4.4175)
Validation loss =  tensor(4.3839)
Validation loss =  tensor(4.3647)
Validation loss =  tensor(4.4633)
Validation loss =  tensor(4.3761)
Validation loss =  tensor(4.4852)
Validation loss =  tensor(4.3957)
Validation loss =  tensor(4.3795)
Validation loss =  tensor(4.4994)
Validation loss =  tensor(4.4984)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


In [137]:
 # testing method for trainer
trainer.test(clfn_basic_cnn, Food101)

[-INFORMATION-] load - FOOD 101 dataset...
[-INFORMATION-] data loaded successful


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         4.0831685066223145
        test_loss            4.402837753295898
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 4.0831685066223145, 'test_loss': 4.402837753295898}]

# ALL CONVOLUTIONAL NETWORKS

In [138]:
from torch import nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.optim import SGD

def acc(output, target, topk=(1,)):
    """generating the specified value of k """
    max_a = max(topk)
    size_batch = target.size(0)

    _, pred_a = output.topk(max_a, 1, True, True)
    pred_a = pred_a.t()
    just = pred_a.eq(target.view(1, -1).expand_as(pred_a))
    res_a = []

    for k in topk:
        just_k = just[:k].view(-1).float().sum(0)
        res_a.append(just_k.mul_(100.0 / size_batch))
    return res_a

class all_con_(pl.LightningModule):
    def __init__(self):
          # Defining the model
        super(all_con_,self).__init__()
          # 1st  2D-convolutional-layer has input  32 is output of feature size and kernel size being 3
        
        self.con_1 = nn.Conv2d(3, 32, 3)
          # 2nd  2D-convolutional-layer has input 32 from the above layer
        self.con_2 = nn.Conv2d(32, 64, 3)
          # 3rd  2D-convolutional-layer has input 32 from above
        #self.con_3 = nn.Conv2d(64, 101, 3)  
          # 1st layer connects
        self.fc_1 = nn.Linear(5760,256)

         # 2nd layer connects
        self.fc_2 = nn.Linear(256,128)

         # 3rd layer connects gives 101 output 

        self.out = nn.Linear(128,101)     
        self.lr = 0.01
         # applying cross entropy
        self.loss = nn.CrossEntropyLoss()


        
                 
    def forward(self, x):
        # defining the forward pass
        size_batch, _, _, _ = x.size()
        x = F.relu(self.con_1(x))
        x = F.relu(self.con_2(x))
        #x = F.relu(self.con_3(x))
        x = F.max_pool2d(x, 2) 
        x = x.view(size_batch,-1)
        #x = F.relu(self.fc_1(x))
        #x = F.relu(self.fc_2(x))
        #x = F.relu(self.fc_3(x))
        x = x.flatten(1)
        return self.out(x)
    
    def configure_optimizers(self):
         #  Optimizer being defined
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # training loop defined
        a, b = train_batch
        logits = self.forward(a)
        train_loss = self.loss(logits,b)
        print("Training loss = ", train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # validation loops defined
        a, b = valid_batch
        logits = self.forward(a)
        valid_loss = self.loss(logits,b)
        print("Validation loss = ", valid_loss)
        valid_pred = acc(logits, b)
        self.log("val_accuracy", valid_pred[0])
        self.log("val_loss", valid_loss)
        return valid_loss
    
    def test_step(self, test_batch, batch_idx):
         # testing steps being defined
        l_test=[]
        a, b = test_batch
        logits = self.forward(a)
        test_loss = self.loss(logits,b)
        l_test.append(test_loss)
        test_pred = acc(logits, b)
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [140]:
 # model object being created
clfn_all_con = all_con_()
 #data module object created
#Food101 = Food101()
 # trainer object being created
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [142]:
 # fit method for trainer
trainer.fit(clfn_all_con, Food101)


[-INFORMATION-] load - FOOD 101 dataset...



  | Name  | Type             | Params
-------------------------------------------
0 | con_1 | Conv2d           | 896   
1 | con_2 | Conv2d           | 18.5 K
2 | fc_1  | Linear           | 14.7 M
3 | fc_2  | Linear           | 32.9 K
4 | out   | Linear           | 13.0 K
5 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
14.8 M    Trainable params
0         Non-trainable params
14.8 M    Total params
59.245    Total estimated model params size (MB)


[-INFORMATION-] data loaded successful


Sanity Checking: 0it [00:00, ?it/s]

Validation loss =  tensor(4.4378)
Validation loss =  tensor(4.2513)


Training: 0it [00:00, ?it/s]

Training loss =  tensor(4.3980, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.3640, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.3348, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.2564, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.4906, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.3551, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5010, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.3156, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.1725, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.4983, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.4661, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5495, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.4909, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.1851, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.4160, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.2904, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.2189, grad_fn=

Validation: 0it [00:00, ?it/s]

Validation loss =  tensor(4.3079)
Validation loss =  tensor(4.4403)
Validation loss =  tensor(4.2775)
Validation loss =  tensor(4.4381)
Validation loss =  tensor(4.2577)
Validation loss =  tensor(4.3448)
Validation loss =  tensor(4.0228)
Validation loss =  tensor(4.4740)
Validation loss =  tensor(4.3529)
Validation loss =  tensor(4.1747)
Validation loss =  tensor(4.3142)
Validation loss =  tensor(4.6856)
Validation loss =  tensor(4.3232)
Validation loss =  tensor(4.1730)
Validation loss =  tensor(4.0443)
Validation loss =  tensor(4.2290)
Validation loss =  tensor(4.4914)
Validation loss =  tensor(4.4042)
Validation loss =  tensor(4.2645)
Validation loss =  tensor(4.6664)
Validation loss =  tensor(4.3804)
Validation loss =  tensor(4.3046)
Validation loss =  tensor(4.4344)
Validation loss =  tensor(4.2627)
Validation loss =  tensor(4.4475)
Validation loss =  tensor(4.4638)
Validation loss =  tensor(4.3211)
Validation loss =  tensor(4.5835)
Validation loss =  tensor(4.1874)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


In [144]:
 # test method for trainer
trainer.test(clfn_all_con, Food101)

[-INFORMATION-] load - FOOD 101 dataset...
[-INFORMATION-] data loaded successful


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          5.164356231689453
        test_loss           4.3204827308654785
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 5.164356231689453, 'test_loss': 4.3204827308654785}]

# Regularization

In [145]:
def acc(output, target, topk=(1,)):
    """generating the specified value of k """
    max_a = max(topk)
    size_batch = target.size(0)

    _, pred_a = output.topk(max_a, 1, True, True)
    pred_a = pred_a.t()
    just = pred_a.eq(target.view(1, -1).expand_as(pred_a))
    res_a = []

    for k in topk:
        just_k = just[:k].view(-1).float().sum(0)
        res_a.append(just_k.mul_(100.0 / size_batch))
    return res_a

class regularization_(pl.LightningModule):
    def __init__(self):
          # Defining the model
        super(regularization_,self).__init__()
          # 1st  2D-convolutional-layer has input  32 is output of feature size and kernel size being 3
        
        self.con_1 = nn.Conv2d(3, 32, 3)
          # 2nd  2D-convolutional-layer has input 32 from the above layer
        self.con_2 = nn.Conv2d(32, 64, 3)
        
         # 1st layer connects
        self.fc_1 = nn.Linear(57600,256)

         # 2nd layer connects
        self.fc_2 = nn.Linear(256,128)

         # 3rd layer connects gives 101 output 
        self.out = nn.Linear(128,101)
         # proportion defining for dropout
        self.dropout = nn.Dropout(0.25)
        self.lr = 0.01
         # applying cross entropy
        self.loss = nn.CrossEntropyLoss()
          
           
    def forward(self, x):
        # defining the forward pass
        size_batch, _, _, _ = x.size()
        x = F.relu(self.con_1(x))
        x = F.relu(self.con_2(x))
        x = F.max_pool2d(x, 2) 
        x = x.view(size_batch,-1)
        x = F.relu(self.fc_1(x))
        x = self.dropout(x)
        x = F.relu(self.fc_2(x))
        return self.out(x)
        
    
    def configure_optimizers(self):
         #  Optimizer being defined
        return SGD(self.parameters(),lr = self.lr)
       
    def training_step(self, train_batch, batch_idx):
        # training loop defined
        a, b = train_batch
        logits = self.forward(a)
        train_loss = self.loss(logits,b)
        print("Training loss = ", train_loss)
        self.log("train_loss", train_loss)
        return train_loss
        
    def validation_step(self, valid_batch, batch_idx):
        # validation loops defined
        a, b = valid_batch
        logits = self.forward(a)
        valid_loss = self.loss(logits,b)
        print("Validation loss = ", valid_loss)
        valid_pred = acc(logits, b)
        self.log("val_accuracy", valid_pred[0])
        self.log("val_loss", valid_loss)
        return valid_loss
    
    def test_step(self, test_batch, batch_idx):
         # testing steps being defined
        l_test=[]
        a, b = test_batch
        logits = self.forward(a)
        test_loss = self.loss(logits,b)
        l_test.append(test_loss)
        test_pred = acc(logits, b)
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]

In [146]:
import pytorch_lightning as pl
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import torch
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

class augmen_Food101(pl.LightningDataModule):
    def prepare_data(self):
        transform=transforms.Compose([
            transforms.RandomResizedCrop(128),
            transforms.RandomCrop(64),
            transforms.RandAugment(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.229, 0.224, 0.225])])
        
        # Preparing data
        print("[-INFORMATION-] load - FOOD 101 dataset...")

      # spliting the train_data, val_data, test_data
        self.train_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="train", download=True, transform=transform)
        self.val_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="test", download=True, transform=transform)
        self.test_data = datasets.Food101('/C/Users/syeda/Desktop/Machine Learning/Assignment-6', split="test", download=True, transform=transform)
        print('[-INFORMATION-] data loaded successful')
     #defining train data loader
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size= 32, shuffle=True, num_workers=8)
     #defining value data loader
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size= 32, shuffle=True, num_workers=8)
     #defining test data loader
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size= 32, shuffle=True, num_workers=8)

In [147]:
 # model object being created
clfn_regularization_ = regularization_()
# data module object created
sep_Food101 = augmen_Food101()
 # trainer object being created
trainer = pl.Trainer(accelerator ="cpu", max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [148]:
 # fit method for trainer
trainer.fit(clfn_regularization_, augmen_Food101)

[-INFORMATION-] load - FOOD 101 dataset...



  | Name    | Type             | Params
---------------------------------------------
0 | con_1   | Conv2d           | 896   
1 | con_2   | Conv2d           | 18.5 K
2 | fc_1    | Linear           | 14.7 M
3 | fc_2    | Linear           | 32.9 K
4 | out     | Linear           | 13.0 K
5 | dropout | Dropout          | 0     
6 | loss    | CrossEntropyLoss | 0     
---------------------------------------------
14.8 M    Trainable params
0         Non-trainable params
14.8 M    Total params
59.245    Total estimated model params size (MB)


[-INFORMATION-] data loaded successful


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\syeda\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


...

Training loss =  tensor(4.6229, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6225, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6229, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6295, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6223, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6090, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6165, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6092, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6250, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5963, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6106, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6007, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5894, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6062, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6110, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.6280, grad_fn=<NllLossBackward0>)
Training loss =  tensor(4.5849, grad_fn=

Validation: 0it [00:00, ?it/s]

Validation loss =  tensor(4.5949)
Validation loss =  tensor(4.4941)
Validation loss =  tensor(4.6055)
Validation loss =  tensor(4.5227)
Validation loss =  tensor(4.5772)
Validation loss =  tensor(4.3825)
Validation loss =  tensor(4.5932)
Validation loss =  tensor(4.5089)
Validation loss =  tensor(4.6069)
Validation loss =  tensor(4.4948)
Validation loss =  tensor(4.4931)
Validation loss =  tensor(4.5910)
Validation loss =  tensor(4.5483)
Validation loss =  tensor(4.5964)
Validation loss =  tensor(4.5519)
Validation loss =  tensor(4.5615)
Validation loss =  tensor(4.4835)
Validation loss =  tensor(4.5603)
Validation loss =  tensor(4.5562)
Validation loss =  tensor(4.4977)
Validation loss =  tensor(4.5796)
Validation loss =  tensor(4.5061)
Validation loss =  tensor(4.5921)
Validation loss =  tensor(4.5252)
Validation loss =  tensor(4.5067)
Validation loss =  tensor(4.5358)
Validation loss =  tensor(4.5196)
Validation loss =  tensor(4.5631)
Validation loss =  tensor(4.5485)
Validation los

`Trainer.fit` stopped: `max_epochs=1` reached.


tensor(4.0449)


In [149]:
 # test method for trainer
trainer.test(clfn_regularization_, augmen_Food101)

[-INFORMATION-] load - FOOD 101 dataset...
[-INFORMATION-] data loaded successful


c:\Users\syeda\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:488: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         2.8237624168395996
        test_loss            4.536500930786133
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 2.8237624168395996, 'test_loss': 4.536500930786133}]

# Transfer Learning

In [160]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import Food101
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl


def acc(output, target, topk=(1,)):
    """generating the specified value of k """
    max_a = max(topk)
    size_batch = target.size(0)

    _, pred_a = output.topk(max_a, 1, True, True)
    pred_a = pred_a.t()
    just = pred_a.eq(target.view(1, -1).expand_as(pred_a))
    res_a = []

    for k in topk:
        just_k = just[:k].view(-1).float().sum(0)
        res_a.append(just_k.mul_(100.0 / size_batch))
    return res_a

class Imagenet_Transfer_Learning(pl.LightningModule):
    def __init__(self, data_path, batch_size, lr):
        super().__init__()
        
        self.data_pth = data_path
        self.size_batch = batch_size
        self.lr = lr
        
        # Preparing data
        data_st = Food101(data_path, transform=transforms.Compose([transforms.RandAugment(),transforms.RandomCrop(56),transforms.ToTensor(),]), download=True)

        data_set_size = len(data_st)
        train_size = int(data_set_size * .95)
        val_size = data_set_size - train_size
        self.train_data_set, self.val_data_set = torch.utils.data.random_split(data_st, [train_size, val_size])

         # deriving loss function
        self.loss_fn = nn.CrossEntropyLoss()
        
         # pretrained resnet being initialized
        back_bone = models.resnet18(pretrained=True)
        numbr_filters = back_bone.fc.in_features
        layers = list(back_bone.children())[:-1]
        self.featr_extractor = nn.Sequential(*layers)

        # food101 being classified using pretrained model
        numbr_targ_classes = 101
        self.clasfr = nn.Linear(numbr_filters, numbr_targ_classes)
     #defining forward method
    def forward(self, x):
        self.featr_extractor.eval()
        with torch.no_grad():
            represent = self.featr_extractor(x).flatten(1)
        x = self.clasfr(represent)
        
        return x
      #defining training step  
    def training_step(self, batch, batch_idx):
        input, targ = batch
        output = self(input)
        loss = self.loss_fn(output, targ)
        self.log("train_loss", loss)
        
        return loss
        # defining validation step
    def validation_step(self, batch, batch_idx):
        input, targ = batch
        output = self(input)
        loss = self.loss_fn(output, targ)
        self.log("val_loss", loss)
        # defining optimizer
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)
        #defining train data loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_data_set,batch_size=self.size_batch,num_workers=8,shuffle=True)
        #defining val data loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data_set,batch_size=self.size_batch,num_workers=8,shuffle=False)
        #defining test data loader
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data_set,batch_size=self.size_batch,num_workers=8,shuffle=False)
     #defining test step
    def test_step(self, test_batch, batch_idx):
        l_test=[]
        a, b = test_batch
        logits = self.forward(a)
        test_loss = self.loss_fn(logits,b)
        l_test.append(test_loss)
        test_pred = acc(logits, b)
        self.log("test_accuracy", test_pred[0])
        self.log("test_loss", test_loss)
        return l_test[-1]    

In [161]:
 # calling checkpoint
chkpnt_callbk = ModelCheckpoint(
    monitor="val_loss",
    mode="min")

In [162]:
model = Imagenet_Transfer_Learning("/C/Users/syeda/Desktop/Machine Learning/Assignment-6", 32, 1e-3)

c:\Users\syeda\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\syeda\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [163]:
 # training checkpoint
trainer = pl.Trainer(accelerator="cpu", callbacks=[chkpnt_callbk], max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [164]:
 # fit method for model
trainer.fit(model)


  | Name            | Type             | Params
-----------------------------------------------------
0 | loss_fn         | CrossEntropyLoss | 0     
1 | featr_extractor | Sequential       | 11.2 M
2 | clasfr          | Linear           | 51.8 K
-----------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.913    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [165]:
 # testing the model
trainer.test(model)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          6.362196445465088
        test_loss            4.47664737701416
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 6.362196445465088, 'test_loss': 4.47664737701416}]